In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import spacy
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', -1)

/var/folders/zf/45wdmkkn2358rlr4dgrc7sx00000gn/T/ipykernel_9026/197519883.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df=pd.read_csv('Reviews.csv', sep=',', quotechar='"')

In [3]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' ""The Lion, The Witch, and The Wardrobe"" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if not better than resturants I have eaten at..My husband loved it..will find other recipes to use this in..
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,"I'm disappointed with the flavor. The chocolate notes are especially weak. Milk thickens it but the flavor still disappoints. This was worth a try but I'll never buy again. I will use what's left, which will be gone in no time thanks to the small cans."
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 of those in one training session. I tried to train our dog with ""Ceaser dog treats"", it just made our puppy hyper. If you compare the ingredients, you will know why. Little stars has just basic food ingredients without any preservatives and food coloring. Sweet potato flavor also did not make my hand smell like dog food."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rewarding your dog for being good while grooming. Lower in calories and loved by all the doggies. Sweet potatoes seem to be their favorite Wet Noses treat!


In [4]:
df.Text

0         I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.                                                                                                                                                                                                                                                      
1         Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".                                                                                                                                                                                                                                                                                        

In [5]:
df.dropna(axis = 0, how ='any',inplace=True) 

In [6]:
def clean_text(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation}
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit())])# and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [7]:
df.Text = df.Text.apply(clean_text)

In [8]:
sample_list = df.Text.tolist()
for_end = df.Text.tolist()
sample_list

['i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most',
 'product arrived labeled as jumbo salted peanutsthe peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo',
 'this is a confection that has been around a few centuries it is a light pillowy citrus gelatin with nuts in this case filberts and it is cut into tiny squares and then liberally coated with powdered sugar and it is a tiny mouthful of heaven not too chewy and very flavorful i highly recommend this yummy treat if you are familiar with the story of cs lewis the lion the witch and the wardrobe this is the treat that seduces edmund into selling out his brother and sisters to the witch',
 'if you are looking for the secret ingredient in robitussi

In [9]:
from nltk.corpus import stopwords

en = spacy.load('en_core_web_sm')
stpwrds = en.Defaults.stop_words
all_stopwords = stpwrds.union(stopwords.words('english'))
def remove_stopwords(text):
    finalList = []
    for i in text:
        textArr = i.split(' ')    
        rem_text = ' '.join([i for i in textArr if i not in all_stopwords])
        finalList.append(rem_text)
    return finalList


# remove stopwords from the text
sample_list = remove_stopwords(sample_list)
sample_list

['bought vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better',
 'product arrived labeled jumbo salted peanutsthe peanuts actually small sized unsalted sure error vendor intended represent product jumbo',
 'confection centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story cs lewis lion witch wardrobe treat seduces edmund selling brother sisters witch',
 'looking secret ingredient robitussin believe found got addition root beer extract ordered good cherry soda flavor medicinal',
 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal',
 'got wild hair taffy ordered pound bag taffy enjoyable flavors watermelon root beer melon peppermint grape etc complaint bit redblack licoriceflavored pieces particular favorites kids husband lasted

In [10]:
"""Returns lemmatized text"""

nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
def lemmatize(text):
    #text_list = text.tolist()
    splem = []
    wordlist = []
    for i in range(1000):
        wordlist.append(text[i])
        lst = word_tokenize(text[i])
        #print(lst)
        #wordlist.append(lst)
        doc = nlp(' '.join(map(str,lst)))
        lem = " ".join([token.lemma_ for token in doc])
        #print(lem)
        splem.append(lem)
    #return(' '.join(map(str,splem)))
    return splem

In [11]:
lemmatized = lemmatize(sample_list)
lemmatized

['buy vitality can dog food product find good quality product look like stew process meat smell well labrador finicky appreciate product well',
 'product arrive label jumbo salt peanutsthe peanut actually small sized unsalted sure error vendor intend represent product jumbo',
 'confection century light pillowy citrus gelatin nuts case filbert cut tiny square liberally coat powder sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story cs lewis lion witch wardrobe treat seduce edmund sell brother sister witch',
 'look secret ingredient robitussin believe find get addition root beer extract order good cherry soda flavor medicinal',
 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal',
 'get wild hair taffy order pound bag taffy enjoyable flavor watermelon root beer melon peppermint grape etc complaint bit redblack licoriceflavore piece particular favorite kid husband last week recommend brand taffy delightful treat',
 'salt

In [12]:
fnl_complaints = remove_stopwords(lemmatized)
fnl_complaints

['buy vitality dog food product find good quality product look like stew process meat smell labrador finicky appreciate product',
 'product arrive label jumbo salt peanutsthe peanut actually small sized unsalted sure error vendor intend represent product jumbo',
 'confection century light pillowy citrus gelatin nuts case filbert cut tiny square liberally coat powder sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story cs lewis lion witch wardrobe treat seduce edmund sell brother sister witch',
 'look secret ingredient robitussin believe find addition root beer extract order good cherry soda flavor medicinal',
 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal',
 'wild hair taffy order pound bag taffy enjoyable flavor watermelon root beer melon peppermint grape etc complaint bit redblack licoriceflavore piece particular favorite kid husband week recommend brand taffy delightful treat',
 'saltwater taffy great flavor so

In [13]:
X_train, X_hold = train_test_split(fnl_complaints, test_size=0.3,random_state=10)

In [14]:
X_train

['work college financial aid bowl desk student buy grocery store learn online 3 lb bag think good deal find great deal',
 'potato chip delicious potato chip approximately year difficult find organic chip amazon lower price finally chance order definitely order family love pure void artificial flavoring true flavor present satisifie need salty sweet spicy crunchy',
 'kettle chip careful flavor nauseate fantastic plain chip thick cut potato fry dark golden brown crunchy lightly salt sea salt recommend chip regret itbr br people burn website quick look immediate difference kettle brand ® potato chip beautiful tawny gold cook natural sugar select potato caramelize create chip light gold deep amber result flavor deep rich color artisanal display bagbr br chip oily true explain website kettle food exclusively use expeller press high monounsaturated sunflower andor safflower oil kettle brand ® product oil naturally free trans fatty acids hydrogenate way extra step send product independent par

In [15]:
stemmer = nltk.stem.SnowballStemmer('english')
stopwords = nltk.corpus.stopwords.words('english')

In [16]:
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))) > 3) ] 
    #tokens = map(str.lower, tokens)
    tokens = [w.lower for w in tokens]
    #stems = [stemmer.stem(item) for item in tokens if (item not in stopwords)]
    #return stems
    return tokens

In [17]:
vectorizer = TfidfVectorizer() #, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
tfidf_vectors = vectorizer.fit_transform(X_train)

In [18]:
tfidf_vectors.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
vectorizer.get_feature_names()[-10:]

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['zealand',
 'zen',
 'zero',
 'zinc',
 'zip',
 'ziplock',
 'zippy',
 'zola',
 'zucchini',
 'îts']

In [20]:
clf = decomposition.NMF(n_components=5, random_state=100)

W1 = clf.fit_transform(tfidf_vectors)
H1 = clf.components_

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [21]:
H1

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00062573, 0.00062573, 0.00392783, ..., 0.00102246, 0.        ,
        0.        ],
       [0.00059924, 0.00059924, 0.        , ..., 0.00021467, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0004769 , 0.0004769 , 0.008889  , ..., 0.        , 0.01770425,
        0.0291902 ]])

In [22]:
W1

array([[0.        , 0.0789219 , 0.        , 0.11237   , 0.        ],
       [0.15022228, 0.03044924, 0.00091401, 0.01026524, 0.00522742],
       [0.20086551, 0.01388162, 0.        , 0.        , 0.02060383],
       ...,
       [0.        , 0.06356545, 0.00243719, 0.        , 0.00155992],
       [0.        , 0.        , 0.        , 0.        , 0.13763905],
       [0.        , 0.13926014, 0.        , 0.        , 0.        ]])

In [23]:
num_words=15

vocab = np.array(vectorizer.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
topics

['chip kettle potato salt flavor vinegar spicy brand like good try taste crunchy sea love',
 'good product great flavor love buy taste sugar use like find order price store sauce',
 'tea coffee drink taste morning green like br try cup blend black day smell kind',
 'bag open eat large chip box find small hole size need hard buy way coffee',
 'food dog eat baby cat love like try br month natural old balance daughter allergy']

In [25]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [26]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.00,0.08,0.00,0.11,0.00,3
Doc1,0.15,0.03,0.00,0.01,0.01,0
Doc2,0.20,0.01,0.00,0.00,0.02,0
Doc3,0.00,0.02,0.00,0.03,0.04,4
Doc4,0.01,0.11,0.00,0.00,0.00,1
...,...,...,...,...,...,...
Doc695,0.01,0.01,0.01,0.02,0.01,3
Doc696,0.26,0.00,0.00,0.00,0.00,0
Doc697,0.00,0.06,0.00,0.00,0.00,1
Doc698,0.00,0.00,0.00,0.00,0.14,4


In [27]:
X_train[:5]

['work college financial aid bowl desk student buy grocery store learn online 3 lb bag think good deal find great deal',
 'potato chip delicious potato chip approximately year difficult find organic chip amazon lower price finally chance order definitely order family love pure void artificial flavoring true flavor present satisifie need salty sweet spicy crunchy',
 'kettle chip careful flavor nauseate fantastic plain chip thick cut potato fry dark golden brown crunchy lightly salt sea salt recommend chip regret itbr br people burn website quick look immediate difference kettle brand ® potato chip beautiful tawny gold cook natural sugar select potato caramelize create chip light gold deep amber result flavor deep rich color artisanal display bagbr br chip oily true explain website kettle food exclusively use expeller press high monounsaturated sunflower andor safflower oil kettle brand ® product oil naturally free trans fatty acids hydrogenate way extra step send product independent par

In [28]:
WHold = clf.transform(vectorizer.transform(X_hold[:5]))

In [29]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold[:5]))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [30]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.00,0.03,0.14,0.01,0.00,2
Doc1,0.00,0.05,0.20,0.01,0.00,2
Doc2,0.09,0.04,0.01,0.01,0.00,0
Doc3,0.00,0.05,0.04,0.03,0.01,1
Doc4,0.00,0.07,0.00,0.00,0.00,1


In [31]:
X_hold[:5]

['read review purchase purchase numi tea generally happy theri product',
 'purchase tea neighbor present look sassafras tea long time realize hard find absolutely love great surprise christmas gift',
 'absolutely forget confirm reviewer chip total garbage like chew styrofoam packaging peanut positively awful hyperbole exaggeration ill buy kettle brand reportedly great premium brand literally mass market chip I try taste stale rancid taste virtually salty taste whatsoever completely awful',
 'nt wait try cinnamon coffee cake fresh egg creamy sour cream madagascar bourbon vanilla fill cinnamon streusel cinnamon coffee cake available online fair share cinnamon coffee cake let tell fantasicake cinnamon coffee cake cake size price quality tastebr cake arrive fast carefully package arrive perfect condition come large tub white icing drizzle serve believe large size actually feed family enjoy breakfast day cake thick solid sponge cake type consistency moist fill flavor die sticky cinnamon out